# Xử lí dữ liệu thiếu

Sau khi đã làm sạch, chuẩn hóa định dạng cần tiến hành điền dữ liệu bị thiếu trước khi encoding

# 0. Nguồn dữ liệu

In [110]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/interim/normalize_interim.csv')

## 1. Quy chuẩn các dữ liệu thiếu

In [111]:
vals_to_replace = ['Đang cập nhật', 'null', '', ' ']
df = df.replace(vals_to_replace, np.nan)

## 2. Thống kê dữ liệu thiếu

In [112]:
missing_count = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_data = pd.concat([missing_count, missing_percent], axis=1, keys=['Total Missing', 'Percent'])
print(missing_data[missing_data['Total Missing'] > 0])

          Total Missing    Percent
odometer           3124  20.927117
origin             1095   7.335209
style               840   5.627010
seats               928   6.216506
location            164   1.098607
gear               5851  39.194802
engine             5560  37.245445


## 3. Xử lý dữ liệu thiếu

### 1. Xử lý nhóm số (numerical)

Odometer: 
* Điền mising value là Median 
* Tạo một cột đánh dấu rằng đây chỉ là một giá trị ước lượng, không phải giá trị thực tế

In [113]:
df['odometer_is_missing'] = df['odometer'].isnull().astype(int) # Tạo feature mới
df['odometer'] = df['odometer'].fillna(df['odometer'].median())

Seats: 
Điền Median

In [114]:
df['seats'] = df['seats'].fillna(df['seats'].median())

### 2. Xử lý nhóm phân loại (categorical)

#### 1. Nhóm có tỉ lệ thiếu cao (gear, engine)
Điền `Unknown` để tạo nhóm riêng

In [115]:
cols_high_missing = ['gear', 'engine']
for col in cols_high_missing:
    df[col] = df[col].fillna('Unknown')

#### 2. Nhóm có tỉ lệ thiếu thấp (origin, style, location)
Điền giá trị phổ biến nhất (`Mode`)

In [116]:
cols_low_missing = ['origin', 'style', 'location']
for col in cols_low_missing:
    # Lấy giá trị xuất hiện nhiều nhất (index 0)
    most_frequent = df[col].mode()[0]
    df[col] = df[col].fillna(most_frequent)

### 3. Kiểm tra lại dữ liệu thiếu sau khi xử lý

In [117]:
print(df.isnull().sum())

price                  0
odometer               0
origin                 0
status                 0
brand                  0
series                 0
year                   0
fuel_type              0
style                  0
seats                  0
location               0
gear                   0
engine                 0
odometer_is_missing    0
dtype: int64


# Lưu thay đổi

In [118]:
csv_file = '../data/interim/fill_interim.csv'

df.to_csv(csv_file, index=False, encoding='utf-8-sig')
print('Đã lưu file fill_interim.csv')

Đã lưu file fill_interim.csv
